In [1]:
import pandas as pd
import numpy as np


In [38]:
input_path = './data/'
df = pd.read_csv(input_path + 'data.csv', encoding="ISO-8859-1")
print('df dimension:' + str(df.shape))
print('# Duplicates: ' + str(df.duplicated().sum()))
df.head()

df dimension:(541909, 8)
# Duplicates: 5268


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [39]:
## DATA CLEANING ##

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.drop_duplicates(inplace = True)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [40]:
tab_info = pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
null_info = pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values'})
null_perc_info = pd.DataFrame(df.isnull().sum()/df.shape[0]).T.rename(index={0:'null values (%)'})
tab_info = tab_info.append([null_info, null_perc_info])
display(tab_info)
#interesting - 25% of the rows are not associated to any CustomerID (and these rows are then useless)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,int64,datetime64[ns],float64,float64,object
null values,0,0,1454,0,0,0,135037,0
null values (%),0.0,0.0,0.002709,0.0,0.0,0.0,0.251634,0.0


In [41]:
df.dropna(axis = 0, subset = ['CustomerID'], inplace = True)
print('df dimension:', df.shape)
tab_info = pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
null_info = pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values'})
null_perc_info = pd.DataFrame(df.isnull().sum()/df.shape[0]).T.rename(index={0:'null values (%)'})
tab_info = tab_info.append([null_info, null_perc_info])
display(tab_info)

df dimension: (401604, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,int64,datetime64[ns],float64,float64,object
null values,0,0,0,0,0,0,0,0
null values (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
## DATA EXPLORATION ##

#country variable

country_df = df.groupby(['Country', 'CustomerID', 'InvoiceNo']).count()
country_df = country_df.reset_index(drop = False)
countries = country_df['Country'].value_counts() # it counts the unique combinations of Country-CustomerID
countries = countries.reset_index(drop = False)
countries.columns = ['Country', 'CustomerID_Count']
display(countries)


,Country,CustomerID_Count
0,United Kingdom,19857
1,Germany,603
2,France,458
3,EIRE,319
4,Belgium,119
5,Spain,105
6,Netherlands,101
7,Switzerland,71
8,Portugal,70
9,Australia,69


In [85]:
import plotly.express as px

fig = px.scatter_geo(countries, locations = 'Country', locationmode = 'country names', size = 'CustomerID_Count', projection="natural earth")
#enable the row below to drop the UK values
#fig = px.scatter_geo(countries[1:], locations = 'Country', locationmode = 'country names', size = 'CustomerID_Count', projection="natural earth")
fig.show()
# the dataset is dominated by orders made from UK

In [86]:
#customers and products variables
pd.DataFrame([{'products': len(df['StockCode'].value_counts()),    
               'transactions': len(df['InvoiceNo'].value_counts()),
               'customers': len(df['CustomerID'].value_counts()),  
              }], columns = ['products', 'transactions', 'customers'], index = ['quantity'])

,products,transactions,customers
quantity,3684,22190,4372


In [89]:
temp = df.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['InvoiceDate'].count()
nb_products_per_basket = temp.rename(columns = {'InvoiceDate':'Number of products'})
nb_products_per_basket.sort_values('CustomerID')

,CustomerID,InvoiceNo,Number of products
0,12346.0,541431,1
1,12346.0,C541433,1
2,12347.0,537626,31
3,12347.0,542237,29
4,12347.0,549222,24
...,...,...,...
22177,18283.0,557956,76
22186,18283.0,580872,50
22188,18287.0,570715,38
22187,18287.0,554065,29
